# L2: How To Use Structured Outputs

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

In [1]:
!pip install -q python-dotenv==1.0.0 openai==1.66.3 pandas==2.2.3 instructor==1.7.4 outlines==0.2.1 transformers==4.49.0 sentencepiece==0.2.0 datasets==3.3.2



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.4/567.4 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.3/343.3 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.6/345.6 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [4]:
import os
#from helper import get_openai_api_key
from google.colab import userdata
KEY = userdata.get('OPEN_API_KEY')

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>
</div>

In [5]:
from openai import OpenAI

# Instantiate the client
client = OpenAI(
    api_key=KEY
)

## Define structure with Pydantic

In [6]:
# The user class from the slides
from pydantic import BaseModel
from typing import Optional

class User(BaseModel):
    name: str
    age: int
    email: Optional[str] = None

In [7]:
completion = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Make up a user."},
    ],
    response_format=User,
)

In [8]:
user = completion.choices[0].message.parsed
user

User(name='Alice Johnson', age=28, email='alice.johnson@example.com')

## The social media mention structure

In [9]:
from pydantic import BaseModel
from enum import Enum
from typing import List, Optional, Literal
from openai import OpenAI

class Mention(BaseModel):
    # The model chooses the product the mention is about,
    # as well as the social media post's sentiment
    product: Literal['app', 'website', 'not_applicable']
    sentiment: Literal['positive', 'negative', 'neutral']

    # Model can choose to respond to the user
    needs_response: bool
    response: Optional[str]

    # If a support ticket needs to be opened,
    # the model can write a description for the
    # developers
    support_ticket_description: Optional[str]

In [10]:
# Example mentions
mentions = [
    # About the app
    "@techcorp your app is amazing! The new design is perfect",
    # Website is down, negative sentiment + needs a fix
    "@techcorp website is down again, please fix!",
    # Nothing to respond to
    "hey @techcorp you're so evil"
]

In [11]:
def analyze_mention(
    mention: str,
    personality: str = "friendly"
) -> Mention:
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": f"""
                Extract structured information from
                social media mentions about our products.

                Provide
                - The product mentioned (website, app, not applicable)
                - The mention sentiment (positive, negative, neutral)
                - Whether to respond (true/false). Don't respond to
                  inflammatory messages or bait.
                - A customized response to send to the user if we need
                  to respond.
                - An optional support ticket description to create.

                Your personality is {personality}.
            """},
            {"role": "user", "content": mention},
        ],
        response_format=Mention,
    )
    return completion.choices[0].message.parsed

In [12]:
print("User post:", mentions[0])
processed_mention = analyze_mention(mentions[0])
processed_mention

User post: @techcorp your app is amazing! The new design is perfect


Mention(product='app', sentiment='positive', needs_response=True, response="Thank you so much for your kind words! We're thrilled to hear you love the new design. If you have any more feedback, feel free to share!", support_ticket_description=None)

In [13]:
rude_mention = analyze_mention(mentions[0], personality="rude")
rude_mention.response

'Yeah, we know. Thanks for noticing. Try not to get too attached, though.'

In [14]:
mention_json_string = processed_mention.model_dump_json(indent=2)
print(mention_json_string)

{
  "product": "app",
  "sentiment": "positive",
  "needs_response": true,
  "response": "Thank you so much for your kind words! We're thrilled to hear you love the new design. If you have any more feedback, feel free to share!",
  "support_ticket_description": null
}


## You try!

In [15]:
class UserPost(BaseModel):
    message: str

def make_post(output_class):
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": """
                You are a customer of Tech Corp (@techcorp), a company
                that provides an app and a website. Create a small
                microblog-style post to them that sends some kind of
                feedback, positive or negative.
            """},
            {"role": "user", "content": "Please write a post."},
        ],
        response_format=output_class,
    )
    return completion.choices[0].message.parsed

new_post = make_post(UserPost)
new_post

UserPost(message='Just wanted to say how much I love the new features on the Tech Corp app! The user interface is sleek and intuitive, making it so easy to navigate. Keep up the great work! 👍 #TechCorp')

In [16]:
analyze_mention(new_post.message)

Mention(product='app', sentiment='positive', needs_response=True, response="Thanks so much for your kind words! We're thrilled to hear that you're enjoying the new features and find the app easy to navigate. Your feedback motivates us to keep improving! 😊", support_ticket_description=None)

In [17]:
class UserPostWithExtras(BaseModel):
    user_mood: Literal["awful", "bad", "evil"]
    product: Literal['app', 'website', 'not_applicable']
    sentiment: Literal['positive', 'negative', 'neutral']
    internal_monologue: List[str]
    message: str

new_post = make_post(UserPostWithExtras)
new_post

UserPostWithExtras(user_mood='bad', product='app', sentiment='negative', internal_monologue=['This app has been crashing way too often lately.', 'I really want to love it, but the bugs are frustrating my workflow.', 'I hope they fix these issues soon.'], message="Hey @techcorp, I'm experiencing frequent crashes with your app. It's becoming a real hassle. Looking forward to some updates or fixes!")

In [18]:
analyze_mention(new_post.message)

Mention(product='app', sentiment='negative', needs_response=True, response="We're really sorry to hear that you're experiencing crashes with our app! Our team is looking into this issue and we appreciate your feedback. Please make sure you're using the latest version, and let us know if the problem persists. Thank you for your patience!", support_ticket_description='User reports frequent crashes with the app. Follow up with the user regarding updates and fixes.')

## Programming with our mentions

In [20]:
def print_mention(processed_mention, mention):
    # Check if we need to respond
    if processed_mention.needs_response:
        # We need to respond
        print(f"Responding to {processed_mention.sentiment} {processed_mention.product} feedback")
        print(f"  User: {mention}")
        print(f"  Response: {processed_mention.response}")
    else:
        print(f"Not responding to {processed_mention.sentiment} {processed_mention.product} post")
        print(f"  User: {mention}")

    if processed_mention.support_ticket_description:
        print(f"  Adding support ticket: {processed_mention.support_ticket_description}")

In [21]:

# Loop through posts that tagged us and store the results in a list
rows = []
for mention in mentions:
    # Call the LLM to get a Mention object we can program with
    processed_mention = analyze_mention(mention)

    # Print out some information
    print_mention(processed_mention, mention)

    # Convert our processed data to a dictionary
    # using Pydantic tools
    processed_dict = processed_mention.model_dump()

    # Store the original message in the dataframe row
    processed_dict['mention'] = mention
    rows.append(processed_dict)

    print("") # Add separator to make it easier to read

Responding to positive app feedback
  User: @techcorp your app is amazing! The new design is perfect
  Response: Thank you so much for your wonderful feedback! We're thrilled to hear that you love the new design. If you have any suggestions or features you'd like to see in the future, feel free to share!

Responding to negative website feedback
  User: @techcorp website is down again, please fix!
  Response: We're really sorry to hear that you're experiencing issues with our website. Our team is looking into this right now. Thank you for your patience!
  Adding support ticket: User reports website is down, need to investigate and resolve the issue.

Not responding to negative not_applicable post
  User: hey @techcorp you're so evil



In [22]:
import pandas as pd

df = pd.DataFrame(rows)
df

,product,sentiment,needs_response,response,support_ticket_description,mention
0,app,positive,True,Thank you so much for your wonderful feedback!...,None,@techcorp your app is amazing! The new design ...
1,website,negative,True,We're really sorry to hear that you're experie...,"User reports website is down, need to investig...","@techcorp website is down again, please fix!"
2,not_applicable,negative,False,None,None,hey @techcorp you're so evil
